In [424]:
# Generate Multiline Vector from Labelbox Labels

from glob import glob
import pandas as pd
import json
import rasterio as rio
from shapely.geometry import LineString
from shapely.geometry import Polygon
from geopandas import GeoDataFrame
import geopandas as gpd

# Extract Chips

import gdal
import os

# Create NDVI

import numpy as np


# Exporting Intersecting Chips

from osgeo import gdal, osr, ogr
from shutil import copyfile

In [8]:
def add_trailing_slash(path):
    if path[-1] != '/':
        path += '/'
    return path


def create_dir(output_dir):
    # If the output folder doesn't exist, create it
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)

In [373]:
# Add NDVI Band

def add_ndvi(input_dir, output_dir, dtype_1=rio.float32, dtype_2=rio.uint16, windows=False):
    
    input_dir = add_trailing_slash(input_dir)
    output_dir = add_trailing_slash(output_dir)
    
    # If the respective grouping folders are not available 
    create_dir(output_dir)

    input_files = glob(input_dir + '/*.tif')
    total = len(input_files)
    
    for i,file_path in enumerate(input_files,1):
        
        print(f'processing file {i} of {total}', end="\r", flush=True)
        
        with rio.open(file_path, 'r') as r:
            
            descriptions = list(r.descriptions)
            descriptions.append('NDVI')
            rast = r.read().astype(dtype_1)
            nir = r.read(8).astype(dtype_1)
            red = r.read(4).astype(dtype_1)

            # Allow division by zero
            np.seterr(divide='ignore', invalid='ignore')

            # Calculate NDVI
            ndvi = np.zeros(nir.shape, dtype=dtype_1)
            ndvi = ((nir - red) / (nir + red)).astype(dtype_1)
            
            # Rescaling for use in 16bit output
      
            ndvi = (ndvi + 1) * (2**15 - 1)
            
            # Add NDVI band to end of array    
            rast = np.concatenate((rast,[ndvi]),axis=0)
    
            meta = r.meta
            meta.update(
                dtype=dtype_2,
                count=meta["count"] + 1)
            
            
            if windows:
                filename = file_path.split('\\')[-1]
            else:
                filename = file_path.split('/')[-1]
                
                
            output_folder = output_dir + '/'
        
            output_filepath = output_dir + filename
            
            with rio.open(output_filepath, 'w', **meta) as dst:
                
                for i,d in enumerate(descriptions,1):
                    dst.set_band_description(i,d)
                    
                dst.write(rast.astype(dtype_2))


In [374]:
input_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/v12_cloudfreemerge_more_bands/misha_polygons_v12_all_bands/"
output_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/v12_cloudfreemerge_more_bands/misha_polygons_v12_all_bands_NDVI/"

r = add_ndvi(input_dir,output_dir)

In [375]:
# Reproject the files

def convert_rasters(src_dir, dest_dir, epsg_format='EPSG:3257', windows=False):
    """Converts the rasters in the src_dir into a different EPSG format,
    keeping the same folder structure and saving them in the dest_dir."""

    src_dir = add_trailing_slash(src_dir)
    dest_dir = add_trailing_slash(dest_dir)
    
    # If the output folder doesn't exist, create it
    create_dir(dest_dir)

    input_files = glob(src_dir + '/*.tif')
    # Keep track of how many files were converted
    n = 1
    total = len(input_files)
    
    for f in input_files:
        print(f'processing file {n} of {total}', end="\r", flush=True)
        
        n += 1
        
        # The way we've set it up, we save each product into a numbered folder,
        # depending on which layer it's in. To keep this structure, we need to
        # pull out the folder number from the file path.
        # How exactly to do this depends on if you're using Windows or not,
        # since the path conventions are different.
        if windows:
            filename = f.split('\\')[-1]
        else:
            filename = f.split('/')[-1]
            
        # If the respective grouping folders are not available 
        create_dir(dest_dir)
        
        output_filepath = dest_dir + filename


#         Finally, we convert
        converted = gdal.Warp(output_filepath, [f],format='GTiff',
                              dstSRS=epsg_format, resampleAlg='near')
        converted = None
        
    print('Finished')


In [376]:
input_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/v12_cloudfreemerge_more_bands/misha_polygons_v12_all_bands_NDVI/"
output_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/v12_cloudfreemerge_more_bands/misha_polygons_v12_all_bands_reprojected/"

convert_rasters(input_dir, output_dir)

Finishedng file 99 of 99


In [377]:
# Extract Chips from Polygons

def extract_chips(input_dir,output_dir,chip_types_array = [1]):
    
    tile_sizes = [i * 100 for i in chip_types_array]
    
    input_files = glob(f'{input_dir}/*.tif')
    
    for tile_size in tile_sizes:
        
        tile_size_x = tile_size
        tile_size_y = tile_size

        for in_path in input_files:         

            ds = gdal.Open(in_path)
            band = ds.GetRasterBand(1)
            xsize = band.XSize
            ysize = band.YSize

            input_filename = in_path.split("/")[-1]
            image_id = in_path.split("/")[-1].split(".")[0].split("_")[0]
            output_filename = image_id + "_"
            out_path_parent = f'{output_dir}/full_chips/' 
            out_path_id = f'{out_path_parent}/{image_id}/'

            create_dir(out_path_parent)
            create_dir(out_path_id)

            
            out_path_full = f'{output_dir}/full_chips/{image_id}/{tile_size}/'
        
            
            create_dir(out_path_full)


            for i in range(0, xsize, tile_size_x):
                for j in range(0, ysize, tile_size_y):
                    gdal.Translate(str(out_path_full + output_filename + str(i) + "_" + str(j) + ".tif"),in_path,
                                   format="GTiff",srcWin=(str(i),str(j),str(tile_size_x),str(tile_size_y)))


In [380]:
input_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/v12_cloudfreemerge_more_bands/misha_polygons_v12_all_bands_reprojected/"
output_dir = '/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_cloudfreemerge'

extract_chips(input_dir,output_dir,chip_types_array = [1])


In [ ]:
# Generate Multi_Polygon Vector and Export to Shapefile

def generate_labelbox_vectors(input_files_dir=input_dir,tif_files_dir=tif_files_dir,output_dir=output_dir):
    
    input_files_dir = add_trailing_slash(input_files_dir)
    tif_files_dir = add_trailing_slash(tif_files_dir)
    output_base_path = add_trailing_slash(output_dir)
    
    json_list = glob(input_files_dir + "*.json")
    tif_file_list = glob(tif_files_dir + "*.tif")
    
    
    polygon_objs = {}

    for j_file in json_list:
        with open(j_file, 'r') as j:
            labels = json.loads(j.read())
            for objects in labels:

                external_id = objects["External ID"]
                
                try:
                    label = external_id.split("/")[-2]
                    image_id = external_id.split("/")[-1].split("_")[0]
                    
                except:
                    label = external_id.split("_")[1].split(".")[0]
                    image_id = external_id.split("_")[0]
                    
                matching_tif = [i for i in tif_file_list if i.split("/")[-1].split(".")[0].split("_")[0] == image_id][0]
                affine = rio.open(matching_tif).transform
                try:
                    for obj in objects['Label']['objects']:
                        temp_poly_coords = []
                        for coords in obj['polygon']:
                            latlon = affine * (coords["x"],coords["y"])
                            temp_poly_coords.append(latlon)
                        polygon_objs.setdefault(label + "_" + image_id,[]).append(Polygon(temp_poly_coords))
                        
                except:
                    continue

        for geo_label_id in polygon_objs.keys():
            
            geo_label = geo_label_id.split("_")[0]
            
            geo_id = geo_label_id.split("_")[1]
            
            output_label_dir = output_base_path + "/" + geo_label
            
            create_dir(output_label_dir)
            
            output_dir = output_label_dir + "/" + geo_id
            
            create_dir(output_dir)
            
            polygons_df = GeoDataFrame(polygon_objs[geo_label_id])
            polygons_df.rename(columns = {0:"geometry"},inplace=True)
            polygons_df.set_geometry(col='geometry', inplace=True)
            polygons_df.to_file(output_dir + "/" + geo_label_id + ".shp", driver='ESRI Shapefile')


In [ ]:
input_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/vectors/labelbox_raw/"
tif_files_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/v8_dynamic_date_range_v4_native_secondary_sort_using_area/misha_polygons_final_version/"
output_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/vectors/labelbox_vectors"


generate_vectors(input_files_dir=input_dir,tif_files_dir=tif_file_dir,output_dir=output_dir)

# Put QGIS Files in Same Directory Strucutre as Labelbox

In [ ]:
qgis_geos = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/vectors/QGIS_labeling_1/"
output_base_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/vectors/QGIS_labeling_2"

geo_files = glob(qgis_geos + "/" + "*" + "/" + "*")
create_dir(output_base_path)

for geo_file in geo_files:
    geo_label = geo_file.split("/")[-1].split("_")[-1].split(".")[0]
    geo_id = geo_file.split("/")[-2]
    output_label_dir = output_base_path + "/" + geo_label            
    create_dir(output_label_dir)            
    output_dir = output_label_dir + "/" + geo_id
    create_dir(output_dir)
    filename = geo_file.split("/")[-1]
    file_output = output_dir + "/" + filename 
    copyfile(geo_file, file_output)

# Old Labelbox Create Vector Code

In [90]:
# Generate Multiline Vector and Export to Shapefile

csv_loc = "/Users/purgatorid/Documents/GitHub/canopy-gis/data_collection/data/labelled/labelbox_test.csv"
tif_file_list = glob("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops/MSK/Individual_Polygons/TIF/*")
output_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Geometry/test_line_labelling/multilines/"

df = pd.read_csv(csv_loc)


polyline_objs = {}
for parent_index in range(len(df)):
    polyline_list = []
    json_obj = json.loads(df["Label"][parent_index])["objects"]
    image_id = df["External ID"][parent_index].split(".")[0]
    matching_tif = [i for i in tif_file_list if i.split("/")[-1].split(".")[0] == image_id][0]
    affine = rio.open(matching_tif).transform
    for index_2 in range(len(json_obj)):
        temp_list = [] 
        try:
            coords_list = json_obj[index_2]["line"]
            for coords in coords_list:
                latlon = affine * (coords["x"],coords["y"])
                temp_list.append(latlon)
        except:
            continue
            
        polyline_objs.setdefault(image_id,[]).append(LineString(temp_list))
        
for geo_id in polyline_objs.keys():
    multiple_line_df = GeoDataFrame(polyline_objs[geo_id])
    multiple_line_df.rename(columns = {0:"geometry"},inplace=True)
    multiple_line_df.set_geometry(col='geometry', inplace=True)
    multiple_line_df.to_file(output_dir + geo_id + ".shp", driver='ESRI Shapefile')

# Intersection Code

In [111]:
# Export Chips Intersecting with Multiline Vectors


dir_list = glob("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops/MSK/Chips/full/*")
vector_list = glob("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Geometry/test_line_labelling/multilines/*.shp")
output_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v2_Misha/Polygon_Crops/MSK/Chips/yes/"

for vector_path in vector_list:
    vec_id = vector_path.split("/")[-1].split(".")[0]
    vector = ogr.Open(vector_path)
    # Get vector geometry
    layer = vector.GetLayer()
    feature_count = vector.GetLayer().GetFeatureCount()
    for feature_number in range(feature_count):
        feature = layer.GetFeature(feature_number)
        vectorGeometry = feature.GetGeometryRef()
        for in_dir in dir_list:
            tif_id = in_dir.split("/")[-1]
            if tif_id == vec_id:
                raster_list = glob(in_dir + "/*")
                for raster_path in raster_list:
                    raster = gdal.Open(raster_path)
                    # Get raster geometry
                    transform = raster.GetGeoTransform()
                    pixelWidth = transform[1]
                    pixelHeight = transform[5]
                    cols = raster.RasterXSize
                    rows = raster.RasterYSize

                    xLeft = transform[0]
                    yTop = transform[3]
                    xRight = xLeft+cols*pixelWidth
                    yBottom = yTop+rows*pixelHeight

                    ring = ogr.Geometry(ogr.wkbLinearRing)
                    ring.AddPoint(xLeft, yTop)
                    ring.AddPoint(xLeft, yBottom)
                    ring.AddPoint(xRight, yBottom)
                    ring.AddPoint(xRight, yTop)
                    ring.AddPoint(xLeft, yTop)
                    rasterGeometry = ogr.Geometry(ogr.wkbPolygon)
                    rasterGeometry.AddGeometry(ring)

                    # If intersection detected, copy chip to destination directory
                    if rasterGeometry.Intersect(vectorGeometry):
        
                        # Check if directory exists
                        if not os.path.isdir(output_path + tif_id):
                            os.mkdir(output_path + tif_id)

                        out_file = output_path + tif_id + "/" + raster_path.split("/")[-1] + ".tif"
                        copyfile(raster_path,out_file)
                        raster_list.remove(raster_path)

# Sandbox

In [213]:
def generate_labelbox_vectors(input_files_dir=input_dir,tif_files_dir=tif_files_dir,output_dir=output_dir):
    
    input_files_dir = add_trailing_slash(input_files_dir)
    tif_files_dir = add_trailing_slash(tif_files_dir)
    output_base_path = add_trailing_slash(output_dir)
    
    json_list = glob(input_files_dir + "*.json")
    tif_file_list = glob(tif_files_dir + "*.tif")
    
    
    polygon_objs = {}

    for j_file in json_list:
        with open(j_file, 'r') as j:
            labels = json.loads(j.read())
            for objects in labels:

                external_id = objects["External ID"]
                
                try:
                    label = external_id.split("/")[-2]
                    image_id = external_id.split("/")[-1].split("_")[0]
                    
                except:
                    label = external_id.split("_")[1].split(".")[0]
                    image_id = external_id.split("_")[0]
                    
                matching_tif = [i for i in tif_file_list if i.split("/")[-1].split(".")[0].split("_")[0] == image_id][0]
                affine = rio.open(matching_tif).transform
                try:
                    for obj in objects['Label']['objects']:
                        temp_poly_coords = []
                        for coords in obj['polygon']:
                            latlon = affine * (coords["x"],coords["y"])
                            temp_poly_coords.append(latlon)
                        polygon_objs.setdefault(label + "_" + image_id,[]).append(Polygon(temp_poly_coords))
                        
                except:
                    continue

        for geo_label_id in polygon_objs.keys():
            
            geo_label = geo_label_id.split("_")[0]
            
            geo_id = geo_label_id.split("_")[1]
            
            output_label_dir = output_base_path + "/" + geo_label
            
            create_dir(output_label_dir)
            
            output_dir = output_label_dir + "/" + geo_id
            
            create_dir(output_dir)
            
            polygons_df = GeoDataFrame(polygon_objs[geo_label_id])
            polygons_df.rename(columns = {0:"geometry"},inplace=True)
            polygons_df.set_geometry(col='geometry', inplace=True)
            polygons_df.to_file(output_dir + "/" + geo_label_id + ".shp", driver='ESRI Shapefile')


In [214]:
input_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/vectors/labelbox_raw/"
tif_files_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/v8_dynamic_date_range_v4_native_secondary_sort_using_area/misha_polygons_final_version/"
output_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/vectors/labelbox_vectors"


generate_vectors(input_files_dir=input_dir,tif_files_dir=tif_file_dir,output_dir=output_dir)

# Put QGIS Files in Same Directory Strucutre as Labelbox

In [234]:
qgis_geos = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/vectors/QGIS_labeling_1/"
output_base_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/vectors/QGIS_labeling_2"

geo_files = glob(qgis_geos + "/" + "*" + "/" + "*")
create_dir(output_base_path)

for geo_file in geo_files:
    geo_label = geo_file.split("/")[-1].split("_")[-1].split(".")[0]
    geo_id = geo_file.split("/")[-2]
    output_label_dir = output_base_path + "/" + geo_label            
    create_dir(output_label_dir)            
    output_dir = output_label_dir + "/" + geo_id
    create_dir(output_dir)
    filename = geo_file.split("/")[-1]
    file_output = output_dir + "/" + filename 
    copyfile(geo_file, file_output)
    
    

In [115]:
tif_file_list[0].split("/")[-1].split(".")[0].split("_")[0]

'78'

# Reproject Vectors to 3257

In [498]:
in_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/vectors/all_vectors"
out_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/vectors/all_vectors_reprojected"

def reproject_vec(in_dir,out_dir, epsg_source=4326,epsg_output=3257):
    
    in_dir = add_trailing_slash(in_dir)
    
    out_dir = add_trailing_slash(out_dir)
    
    create_dir(out_dir)
    
    parent_dir = out_dir.split("/")[-2]
    
    vec_list = glob(f'{in_dir}/*/*/*.shp')
    
    for vec_path in vec_list:
        
        gdf = gpd.read_file(vec_path)
        
        gdf = gdf.set_crs(epsg=epsg_source)
        
        gdf = gdf.to_crs(epsg=epsg_output)
        
        base_path = f'{"/".join(vec_path.split("/")[:9])}/{parent_dir}'
        
        label_dir = f'{base_path}/{vec_path.split("/")[-3]}'
        
        id_dir = f'{label_dir}/{vec_path.split("/")[-2]}'
        
        create_dir(label_dir)
        
        create_dir(id_dir)
        
        filename = f'{id_dir}/{vec_path.split("/")[-1]}'
        
        gdf.to_file(filename)


In [499]:
reproject_vec(in_dir,out_dir)

# Updated Intersection Code

In [500]:
# Export Chips Intersecting with Multiline Vectors


def cp_intersecting_chips(chip_dir,vector_path,output_path,size_list=[100]):
    
    chip_dir = add_trailing_slash(chip_dir)
    vector_path = add_trailing_slash(vector_path)
    output_path = add_trailing_slash(output_path)
    
    
    vector_list = glob(f'{vector_path}/*/*/*.shp')
    
    for size in size_list:
        chip_dir_list = glob(f'{chips_dir}/*/{size}/')
    
        for vector_path in vector_list:
            vec_id = vector_path.split("/")[-2]
            vec_label =  vector_path.split("/")[-3]
            vector = ogr.Open(vector_path)
            # Get vector geometry
            layer = vector.GetLayer()
            feature_count = vector.GetLayer().GetFeatureCount()
            for feature_number in range(feature_count):
                feature = layer.GetFeature(feature_number)
                vectorGeometry = feature.GetGeometryRef()
                for chip_dir in chip_dir_list:
#                     print(chip_dir)
                    tif_id = chip_dir.split("/")[-3]
                    if tif_id == vec_id:
#                         print(tif_id,vec_id)
                        raster_list = glob(chip_dir + "/*.tif")
                        for raster_path in raster_list:
                            raster = gdal.Open(raster_path)
                            # Get raster geometry
                            transform = raster.GetGeoTransform()
                            pixelWidth = transform[1]
                            pixelHeight = transform[5]
                            cols = raster.RasterXSize
                            rows = raster.RasterYSize

                            xLeft = transform[0]
                            yTop = transform[3]
                            xRight = xLeft+cols*pixelWidth
                            yBottom = yTop+rows*pixelHeight

                            ring = ogr.Geometry(ogr.wkbLinearRing)
                            ring.AddPoint(xLeft, yTop)
                            ring.AddPoint(xLeft, yBottom)
                            ring.AddPoint(xRight, yBottom)
                            ring.AddPoint(xRight, yTop)
                            ring.AddPoint(xLeft, yTop)
                            rasterGeometry = ogr.Geometry(ogr.wkbPolygon)
                            rasterGeometry.AddGeometry(ring)

                            # If intersection detected, copy chip to destination directory
                            if rasterGeometry.Intersect(vectorGeometry):                           
                                # Check if directory exists
                                output_label_dir = f'{output_path}{vec_label}'
                                output_label_size_dir = f'{output_label_dir}/{size}'
                                output_label_size_id_dir = f'{output_label_size_dir}/{vec_id}/' 
                                
                                create_dir(output_label_dir)
                                create_dir(output_label_size_dir)
                                create_dir(output_label_size_id_dir)

                                out_file = output_label_size_id_dir + raster_path.split("/")[-1]
                                copyfile(raster_path,out_file)
#                                 raster_list.remove(raster_path)

In [ ]:
chips_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_cloudfreemerge/full"
vec_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/vectors/all_vectors_reprojected"
output_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_cloudfreemerge/yes"

cp_intersecting_chips(chips_dir,vec_dir,output_path)



In [294]:
chips_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_v8/full"

glob(f'{chips_dir}/*/100/') 

['/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_v8/full/78/100/',
 '/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_v8/full/84/100/',
 '/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_v8/full/48/100/',
 '/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_v8/full/89/100/',
 '/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_v8/full/88/100/',
 '/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_v8/full/82/100/',
 '/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_v8/full/12/100/',
 '/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_v8/full/75/100/',
 '/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_v8/full/76/100/',
 '/Volumes/Lacie/zh